In [ ]:
import pygame, random, sys
from pygame.locals import *

In [ ]:
WINDOWWIDTH = 600
WINDOWHEIGHT = 600
TEXTCOLOR = (255, 255, 255)
BACKGROUNDCOLOR = (0, 0, 0)
FPS = 40
BADDIEMINSIZE = 10
BADDIEMAXSIZE = 40
BADDIEMINSPEED = 1
BADDIEMAXSPEED = 8
ADDNEWBADDIERATE = 6
PLAYER_SPEED = 5

In [ ]:
def terminate():
    pygame.quit()
    sys.exit()

In [ ]:
def waitForPlayerToPressKey():
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                terminate()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE: # pressing escape quits
                    terminate()
                return

In [ ]:
def playerHasHitBaddie(playerRect, baddies):
    for b in baddies:
        if playerRect.colliderect(b['rect']):
            return True
    return False

In [ ]:
def drawText(text, font, surface, x, y):
    textobj = font.render(text, 1, TEXTCOLOR)
    textrect = textobj.get_rect()
    textrect.topleft = (x, y)
    surface.blit(textobj, textrect)

In [ ]:
def showGameOverScreen(l_PLAYERSPEED):
    # Stop the game and show the "Game Over" screen.
    pygame.mixer.music.stop()
    gameOverSound.play()

    # Variable for the number the player is typing.
    text = ''
    while True:
        windowSurface.fill(BACKGROUNDCOLOR)
        drawText('New speed: ' + text, font, windowSurface,
                 (WINDOWWIDTH / 3) - 130, (WINDOWHEIGHT / 3) + 130)
        for event in pygame.event.get():
            if event.type == QUIT:
                terminate()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    terminate()
                if event.key == K_RETURN:
                    if text == '':
                        return l_PLAYERSPEED
                    else:
                        return int(text)
                if event.key == K_BACKSPACE:
                    text = text[:-1]
                else:
                    text += event.unicode
        drawText('GAME OVER', font, windowSurface, (WINDOWWIDTH / 3), (WINDOWHEIGHT / 3))
        drawText('Press Enter to play again', font, windowSurface, (WINDOWWIDTH / 3) - 130, (WINDOWHEIGHT / 3) + 50)
        drawText('or type speed and press Enter.', font, windowSurface,
                 (WINDOWWIDTH / 3) - 130, (WINDOWHEIGHT / 3) + 90)
        pygame.display.update()
        mainClock.tick(FPS)

In [ ]:
def main():
    global windowSurface, font, gameOverSound, playerImage, baddieImage, mainClock, PLAYER_SPEED
    pygame.init()
    mainClock = pygame.time.Clock()
    windowSurface = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    pygame.display.set_caption('Dodger')
    pygame.mouse.set_visible(True)
    font = pygame.font.SysFont(None, 48)
    gameOverSound = pygame.mixer.Sound('gameover.wav')
    pygame.mixer.music.load('background.mid')
    playerImage = pygame.image.load('player.png')
    playerRect = playerImage.get_rect()
    baddieImage = pygame.image.load('baddie.png')
    drawText('Dodger', font, windowSurface, (WINDOWWIDTH / 3), (WINDOWHEIGHT / 3))
    drawText('Press a key to start.', font, windowSurface, (WINDOWWIDTH / 3) - 30, (WINDOWHEIGHT / 3) + 50)
    pygame.display.update()
    waitForPlayerToPressKey()
    topScore = 0
    while True:
        baddies = []
        score = 0
        playerRect.topleft = (WINDOWWIDTH / 2, WINDOWHEIGHT - 50)
        moveLeft = moveRight = moveUp = moveDown = False
        reverseCheat = slowCheat = False
        baddieAddCounter = 0
        pygame.mixer.music.play(-1, 0.0)
        while True:
            score += 1
            for event in pygame.event.get():
                if event.type == QUIT:
                    terminate()
                if event.type == KEYDOWN:
                    if event.key == ord('z'):
                        reverseCheat = True
                    if event.key == ord('x'):
                        slowCheat = True
                    if event.key == K_LEFT or event.key == ord('a'):
                        moveRight = False
                        moveLeft = True
                    if event.key == K_RIGHT or event.key == ord('d'):
                        moveLeft = False
                        moveRight = True
                    if event.key == K_UP or event.key == ord('w'):
                        moveDown = False
                        moveUp = True
                    if event.key == K_DOWN or event.key == ord('s'):
                        moveUp = False
                        moveDown = True
                if event.type == KEYUP:
                    if event.key == ord('z'):
                        reverseCheat = False
                        score = 0
                    if event.key == ord('x'):
                        slowCheat = False
                        score = 0
                    if event.key == K_ESCAPE:
                            terminate()
                    if event.key == K_LEFT or event.key == ord('a'):
                        moveLeft = False
                    if event.key == K_RIGHT or event.key == ord('d'):
                        moveRight = False
                    if event.key == K_UP or event.key == ord('w'):
                        moveUp = False
                    if event.key == K_DOWN or event.key == ord('s'):
                        moveDown = False
                if event.type == MOUSEMOTION:
                    playerRect.move_ip(event.pos[0] - playerRect.centerx, event.pos[1] - playerRect.centery)
            if not reverseCheat and not slowCheat:
                baddieAddCounter += 1
            if baddieAddCounter == ADDNEWBADDIERATE:
                baddieAddCounter = 0
                baddieSize = random.randint(BADDIEMINSIZE, BADDIEMAXSIZE)
                newBaddie = {'rect': pygame.Rect(random.randint(0, WINDOWWIDTH-baddieSize), 0 - baddieSize, baddieSize, baddieSize),
                            'speed': random.randint(BADDIEMINSPEED, BADDIEMAXSPEED),
                            'surface':pygame.transform.scale(baddieImage, (baddieSize, baddieSize)),
                            }
                baddies.append(newBaddie)
            if moveLeft and playerRect.left > 0:
                playerRect.move_ip(-1 * PLAYER_SPEED, 0)
            if moveRight and playerRect.right < (WINDOWWIDTH-20):
                playerRect.move_ip(PLAYER_SPEED, 0)
            if moveUp and playerRect.top > 0:
                playerRect.move_ip(0, -1 * PLAYER_SPEED)
            if moveDown and playerRect.bottom < WINDOWHEIGHT:
                playerRect.move_ip(0, PLAYER_SPEED)

            # Snap player to remain inside window
            if playerRect.left < 0:
                playerRect.left = 0
            if playerRect.right > (WINDOWWIDTH - 20):
                playerRect.right = WINDOWWIDTH
            if playerRect.top < 0:
                playerRect.top = 0
            if playerRect.bottom > WINDOWHEIGHT:
                playerRect.bottom = WINDOWHEIGHT
            pygame.mouse.set_pos(playerRect.centerx, playerRect.centery)
            for b in baddies:
                if not reverseCheat and not slowCheat:
                    b['rect'].move_ip(0, b['speed'])
                elif reverseCheat:
                    b['rect'].move_ip(0, -5)
                elif slowCheat:
                    b['rect'].move_ip(0, 1)
            for b in baddies[:]:
                if b['rect'].top > WINDOWHEIGHT:
                    baddies.remove(b)
            windowSurface.fill(BACKGROUNDCOLOR)
            drawText('Score: %s' % (score), font, windowSurface, 10, 0)
            drawText('Top Score: %s' % (topScore), font, windowSurface, 10, 40)
            windowSurface.blit(playerImage, playerRect)
            for b in baddies:
                windowSurface.blit(b['surface'], b['rect'])
            pygame.display.update()
            if playerHasHitBaddie(playerRect, baddies):
                if score > topScore:
                    topScore = score
                break
            mainClock.tick(FPS)
        new_speed = showGameOverScreen(PLAYER_SPEED)
        PLAYER_SPEED = new_speed
        gameOverSound.stop()

In [ ]:
if __name__ == '__main__':
    main()